# 分析

本当は1日単位でDailyでとって変遷まで見たかったが、
間に合わないのでWeeklyで取るしかない。


## KPT_7_26_08_01

### 文字数

#### Slack

下処理について

* 不要なjsonデータは削除しました
  * 主にトップフォルダの内容


In [32]:
import os
import json

# JSONファイルが格納されているディレクトリのパス
directory = 'material/KPT_7_26_08_01/Slack/'

# 文字数の合計を格納する変数
total_characters = {}

# ディレクトリとそのサブディレクトリ内のすべてのファイルを読み込む
for subdir, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".json"): 
            filepath = subdir + os.sep + filename
            with open(filepath, 'r') as f:
                data = json.load(f)
                
                for message in data:
                    if 'blocks' in message and 'user_profile' in message:
                       author = message['user_profile']['first_name']
                       if author not in total_characters:
                            total_characters[author] = 0
                       for block in message['blocks']:
                          if 'elements' in block:
                            for element in block['elements']:
                              if 'rich_text_section' in element['type']:
                                for element2 in element['elements']:
                                  if 'text' == element2['type']:
                                    text = element2['text']
                                    total_characters[author] += len(text)                                    
                                    
print(f"Total number of characters: {total_characters}")

print(total_characters['Nakayama'])
print(total_characters['satoshi.\u200bhirata'])
print(total_characters['Hibiki'])
print(total_characters['鈴木真希'])
print(total_characters['Dai'])
print(total_characters['Ohno'])
print(total_characters['FABIAN'])


Total number of characters: {'satoshi.\u200bhirata': 450, 'Dai': 2152, 'FABIAN': 753, 'Nakayama': 1016, '鈴木真希': 90, 'Ohno': 3050, 'Hibiki': 2906, 'slackbot': 32}
1016
450
2906
90
2152
3050
753


### 文章量

### 単語のカウント数


#### Slack

In [33]:
import os
import json
import MeCab
import ipadic
from collections import Counter
import re

CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'

# JSONファイルが格納されているディレクトリのパス
directory = 'material/KPT_7_26_08_01/'

# MeCabのインスタンスを作成
m = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)

# 単語のカウントを格納するCounterオブジェクト
word_counts = Counter()

# ディレクトリとそのサブディレクトリ内のすべてのファイルを読み込む
for subdir, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".json"): 
            filepath = subdir + os.sep + filename
            with open(filepath, 'r') as f:
                data = json.load(f)
                for message in data:
                    if 'text' in message:
                        text = message['text']
                        # HTMLエンティティを除外
                        text = re.sub(r'&[a-z]+;', '', text)
                        text = re.sub(r'https*:[a-zA-Z0-9%./?]+', '', text)
                        text = re.sub(r'AT-[0-9A-Za-z]+', '', text)
                        text = re.sub(r'[0-9]+', '', text)
                        parsed_text = m.parse(text).split('\n')
                        words = [word.split('\t')[0] for word in parsed_text 
                                 if word.strip() 
                                 and len(word.split('\t')) > 1 
                                 and (
                                     word.split('\t')[3].startswith('名詞') 
                                     or word.split('\t')[3].startswith('動詞') 
                                     or word.split('\t')[3].startswith('形容詞')
                                 ) 
                                 and len(word.split('\t')[0]) > 1 
                                 ]  
                        # 名詞、動詞、形容詞のみを抽出し、一文字の単語や英字のみの単語を除外
                        word_counts.update(words)

# 最も頻繁に出現する単語の上位3つを出力
top_three = word_counts.most_common(20)
for word, count in top_three:
    print(f'Word: {word}, Count: {count}')

Word: スライド, Count: 85
Word: 作成, Count: 54
Word: さん, Count: 40
Word: id, Count: 34
Word: する, Count: 33
Word: mode, Count: 33
Word: designnode, Count: 32
Word: designt, Count: 32
Word: 発表, Count: 30
Word: 確認, Count: 26
Word: お願い, Count: 25
Word: いる, Count: 24
Word: リモート, Count: 23
Word: こと, Count: 22
Word: よう, Count: 21
Word: コメント, Count: 21
Word: man, Count: 21
Word: でき, Count: 20
Word: Hibiki, Count: 20
Word: Mizuno, Count: 20
